In [8]:
%reload_ext autoreload
%autoreload 2

In [9]:
from result_saver import SaverProvider

provider = SaverProvider()

# Parameters

In [16]:
# Fixed parameters

DEVICE = 'ibm_kyoto'
_RESETS = True

backend = provider.get_backend(DEVICE)

In [17]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]


print("rounds:", rounds)


rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [18]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(plot=False, Ancilla_threshold=0.2)

Trying RepCode string of length 109 => distance 55... Current maxs: CX: 1.0, Ancilla: 1.0
Trying RepCode string of length 107 => distance 54... Current maxs: CX: 1.0, Ancilla: 0.20079999999999998
Trying RepCode string of length 105 => distance 53... Current maxs: CX: 1.0, Ancilla: 0.14470000000000005
Trying RepCode string of length 103 => distance 52... Current maxs: CX: 1.0, Ancilla: 0.14470000000000005
Trying RepCode string of length 101 => distance 51... Current maxs: CX: 1.0, Ancilla: 0.131
Trying RepCode string of length 99 => distance 50... Current maxs: CX: 1.0, Ancilla: 0.131


In [19]:
path_info

{'mean_gate_error': 0.012397755700991539,
 'min_gate_error': 0.003959566073002019,
 'max_gate_error': 0.07846855590325622,
 'mean_readout_error': 0.029825252525252522,
 'min_readout_error': 0.0036000000000000476,
 'max_readout_error': 0.20079999999999998,
 'mean_ancilla_error': 0.02650408163265305,
 'min_ancilla_error': 0.0036000000000000476,
 'max_ancilla_error': 0.131,
 'CX_fidelity': 0.292580773362723}

In [20]:
"0.45279818562360047}"

link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [126, 125, 124, 123, 122, 121, 120, 119, 118, 110, 100, 101, 102, 103, 104, 105, 106, 93, 87, 88, 89, 74, 70, 69, 68, 55, 49, 50, 51, 36, 32, 31, 30, 17, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 35, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 52, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 73, 85, 84, 83, 82, 81, 80, 79, 91, 98, 97, 96, 109, 114, 115, 116]
layout: [125, 123, 121, 119, 110, 101, 103, 105, 93, 88, 74, 69, 55, 50, 36, 31, 17, 11, 9, 7, 5, 3, 1, 14, 19, 21, 23, 25, 27, 35, 46, 44, 42, 40, 38, 52, 57, 59, 61, 63, 65, 73, 84, 82, 80, 91, 97, 109, 115, 126, 124, 122, 120, 118, 100, 102, 104, 106, 87, 89, 70, 68, 49, 51, 32, 30, 12, 10, 8, 6, 4, 2, 0, 18, 20, 22, 24, 26, 28, 47, 45, 43, 41, 39, 37, 56, 58, 60, 62, 64, 66, 85, 83, 81, 79, 98, 96, 114, 116]
distance: 50


# No rounds loop

In [23]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 0

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20
REPETITIONS = 1

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
# qc.delay(unit='s', duration=10*range_rd[1])
# qc.delay(unit='s', duration=120*range_rd[1])
# qc.delay(unit='s', duration=100*range_rd[1])
# qc.delay(unit='s', duration=200*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)



# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', rep_delay=range_rd[1],
                job_tags=[f'10 + rep_delay w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  10


100%|██████████| 1/1 [00:02<00:00,  2.57s/it]


KeyboardInterrupt: 

In [9]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [02:01<00:00,  6.08s/it]


In [10]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [02:01<00:00,  6.06s/it]


In [11]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [01:58<00:00,  5.94s/it]


# TORINO!!!!

In [12]:
# Fixed parameters

DEVICE = 'ibm_torino'
_RESETS = True

backend = provider.get_backend(DEVICE)

In [13]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]

print("rounds:", rounds)

rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [14]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(longest_length=111, Ancilla_threshold=0.2)

Trying RepCode string of length 111 => distance 56... Current maxs: CX: 1.0, Ancilla: 1.0
Trying RepCode string of length 109 => distance 55... Current maxs: CX: 0.18813966976125793, Ancilla: 0.28
Trying RepCode string of length 107 => distance 54... Current maxs: CX: 0.026618840505434227, Ancilla: 0.28
Trying RepCode string of length 105 => distance 53... Current maxs: CX: 0.026618840505434227, Ancilla: 0.28
Trying RepCode string of length 103 => distance 52... Current maxs: CX: 0.026618840505434227, Ancilla: 0.23070000000000002
Trying RepCode string of length 101 => distance 51... Current maxs: CX: 0.026618840505434227, Ancilla: 0.23070000000000002
Trying RepCode string of length 99 => distance 50... Current maxs: CX: 0.016139661932852933, Ancilla: 0.2592


In [15]:
path_info

{'mean_gate_error': 0.006792808977522376,
 'min_gate_error': 0.0015451077598001572,
 'max_gate_error': 0.03213930156287431,
 'mean_readout_error': 0.03306767676767676,
 'min_readout_error': 0.0040000000000000036,
 'max_readout_error': 0.2592,
 'mean_ancilla_error': 0.03696938775510203,
 'min_ancilla_error': 0.010299999999999976,
 'max_ancilla_error': 0.1233333333333333,
 'CX_fidelity': 0.5119213662743483}

In [16]:
link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [37, 52, 51, 50, 49, 48, 47, 46, 55, 65, 66, 67, 68, 69, 70, 71, 75, 90, 89, 88, 87, 86, 85, 84, 93, 103, 104, 105, 106, 107, 108, 109, 113, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 110, 97, 96, 95, 91, 76, 77, 78, 79, 80, 81, 82, 73, 63, 62, 61, 60, 59, 58, 57, 53, 38, 39, 40, 41, 42, 43, 44, 35, 25, 24, 23, 22, 21, 20, 19, 15, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18, 31, 30]
layout: [52, 50, 48, 46, 65, 67, 69, 71, 90, 88, 86, 84, 103, 105, 107, 109, 128, 126, 124, 122, 120, 118, 116, 97, 95, 76, 78, 80, 82, 63, 61, 59, 57, 38, 40, 42, 44, 25, 23, 21, 19, 0, 2, 4, 6, 8, 10, 12, 31, 37, 51, 49, 47, 55, 66, 68, 70, 75, 89, 87, 85, 93, 104, 106, 108, 113, 127, 125, 123, 121, 119, 117, 110, 96, 91, 77, 79, 81, 73, 62, 60, 58, 53, 39, 41, 43, 35, 24, 22, 20, 15, 1, 3, 5, 7, 9, 11, 18, 30]
distance: 50


# No rounds loop

In [17]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 4

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  50


100%|██████████| 20/20 [01:33<00:00,  4.69s/it]


In [18]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [01:32<00:00,  4.65s/it]


In [19]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [01:28<00:00,  4.41s/it]


In [21]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

# Parameters

In [3]:
# Fixed parameters

DEVICE = 'ibm_osaka'
_RESETS = True

backend = provider.get_backend(DEVICE)

In [4]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]


print("rounds:", rounds)


rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [10]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(plot=False, Ancilla_threshold=0.2, longest_length=91, CX_threshold=0.11)

Trying RepCode string of length 91 => distance 46... Current maxs: CX: 1.0, Ancilla: 1.0


In [11]:
path_info

{'mean_gate_error': 0.011911873489514006,
 'min_gate_error': 0.002524520843034589,
 'max_gate_error': 0.10922690058103184,
 'mean_readout_error': 0.032970695970695976,
 'min_readout_error': 0.0033000000000000806,
 'max_readout_error': 0.19320000000000004,
 'mean_ancilla_error': 0.035814814814814806,
 'min_ancilla_error': 0.0033000000000000806,
 'max_ancilla_error': 0.19320000000000004,
 'CX_fidelity': 0.33532047918393754}

In [12]:
"0.45279818562360047}"

link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [117, 116, 115, 114, 109, 96, 95, 94, 90, 75, 76, 77, 71, 58, 57, 56, 52, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 35, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 30, 31, 32, 36, 51, 50, 49, 55, 68, 67, 66, 65, 64, 63, 62, 72, 81, 80, 79, 91, 98, 99, 100, 101, 102, 103, 104, 111, 122, 123, 124, 125, 126, 112, 108, 107]
layout: [116, 114, 96, 94, 75, 77, 58, 56, 37, 39, 41, 43, 45, 47, 28, 26, 24, 22, 20, 18, 0, 2, 4, 6, 8, 10, 12, 30, 32, 51, 49, 68, 66, 64, 62, 81, 79, 98, 100, 102, 104, 122, 124, 126, 108, 117, 115, 109, 95, 90, 76, 71, 57, 52, 38, 40, 42, 44, 46, 35, 27, 25, 23, 21, 19, 14, 1, 3, 5, 7, 9, 11, 17, 31, 36, 50, 55, 67, 65, 63, 72, 80, 91, 99, 101, 103, 111, 123, 125, 112, 107]
distance: 46


# No rounds loop

In [13]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 4

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  50
09:55:44 Warning: updating metadata. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 46


100%|██████████| 20/20 [02:19<00:00,  7.00s/it]


In [14]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [02:18<00:00,  6.93s/it]


In [15]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [02:09<00:00,  6.48s/it]


In [16]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  50


100%|██████████| 20/20 [02:06<00:00,  6.34s/it]


# Parameters

In [12]:
# Fixed parameters

DEVICE = 'ibm_kyoto'
_RESETS = True

backend = provider.get_backend(DEVICE)

Submitting for rounds =  50


NameError: name 'distance' is not defined

In [14]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]


print("rounds:", rounds)


Submitting for rounds =  50


NameError: name 'distance' is not defined

# Find the best layout

In [15]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(plot=False, Ancilla_threshold=0.2, longest_length=103)

Submitting for rounds =  50


NameError: name 'distance' is not defined

In [66]:
path_info

{'mean_gate_error': 0.012397755700991539,
 'min_gate_error': 0.003959566073002019,
 'max_gate_error': 0.07846855590325622,
 'mean_readout_error': 0.03277575757575756,
 'min_readout_error': 0.0038000000000000256,
 'max_readout_error': 0.19799999999999995,
 'mean_ancilla_error': 0.030836054421768698,
 'min_ancilla_error': 0.0042999999999999705,
 'max_ancilla_error': 0.1342000000000001,
 'CX_fidelity': 0.292580773362723}

In [67]:
"0.45279818562360047}"

link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [126, 125, 124, 123, 122, 121, 120, 119, 118, 110, 100, 101, 102, 103, 104, 105, 106, 93, 87, 88, 89, 74, 70, 69, 68, 55, 49, 50, 51, 36, 32, 31, 30, 17, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 35, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 52, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 73, 85, 84, 83, 82, 81, 80, 79, 91, 98, 97, 96, 109, 114, 115, 116]
layout: [125, 123, 121, 119, 110, 101, 103, 105, 93, 88, 74, 69, 55, 50, 36, 31, 17, 11, 9, 7, 5, 3, 1, 14, 19, 21, 23, 25, 27, 35, 46, 44, 42, 40, 38, 52, 57, 59, 61, 63, 65, 73, 84, 82, 80, 91, 97, 109, 115, 126, 124, 122, 120, 118, 100, 102, 104, 106, 87, 89, 70, 68, 49, 51, 32, 30, 12, 10, 8, 6, 4, 2, 0, 18, 20, 22, 24, 26, 28, 47, 45, 43, 41, 39, 37, 56, 58, 60, 62, 64, 66, 85, 83, 81, 79, 98, 96, 114, 116]
distance: 50


# No rounds loop

In [68]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 3

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  40


100%|██████████| 20/20 [01:43<00:00,  5.18s/it]


In [69]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  40


100%|██████████| 20/20 [01:44<00:00,  5.25s/it]


In [70]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  40


100%|██████████| 20/20 [01:51<00:00,  5.58s/it]


In [71]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis, resets=_RESETS)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=120*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1, scheduling_method='alap') 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job 
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info, resets=_RESETS)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'120 del Subset w/ RESET {distance}, {T}, {logical}, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  40


100%|██████████| 20/20 [01:45<00:00,  5.27s/it]
